In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import timeit
import matplotlib.pyplot as plt
from preprocess_func import *
%matplotlib inline


#tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
X,y = preprocess_filtering_data(date='2011_09_26', out_name='train_2', serieses=[1])

1
(3229, 370, 24, 3)
(3229,)
saved


In [3]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [4]:
H=370
W=24
C=3
img_size_flat = H*W
img_shape = (H,W, C)
num_classes = 47

In [5]:
y_hot = np.zeros((y.shape[0], 47))
y_hot[np.arange(y.shape[0]), y] = 1

In [6]:
X.shape

(3229, 370, 24, 3)

In [7]:
X=X.reshape(-1,img_size_flat,C)

In [8]:
X.shape

(3229, 8880, 3)

In [9]:
X_train = X[100:]
y_train = y[100:]

In [10]:
X_test = X[:100]
y_test = y[:100]

In [11]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(X_train)},
    y=np.array(y_train),
    num_epochs=None,
    shuffle=True)

In [12]:

train_input_fn

<function tensorflow.python.estimator.inputs.numpy_io.numpy_input_fn.<locals>.input_fn>

In [13]:
train_input_fn()

({'x': <tf.Tensor 'random_shuffle_queue_DequeueMany:1' shape=(128, 8880, 3) dtype=float32>},
 <tf.Tensor 'random_shuffle_queue_DequeueMany:2' shape=(128,) dtype=int64>)

In [14]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(X_test)},
    y=np.array(y_test),
    num_epochs=1,
    shuffle=False)

In [15]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":np.array(X_test[:20])},
    num_epochs=1,
    shuffle=False)

In [16]:
def model_fn(features, labels, mode, params):
    # Args:
    #
    # features: This is the x-arg from the input_fn.
    # labels:   This is the y-arg from the input_fn,
    #           see e.g. train_input_fn for these two.
    # mode:     Either TRAIN, EVAL, or PREDICT
    # params:   User-defined hyper-parameters, e.g. learning-rate.
    
    # Reference to the tensor named "x" in the input-function.
    x = features["x"]

    # The convolutional layers expect 4-rank tensors
    # but x is a 2-rank tensor, so reshape it.
    net = tf.reshape(x, [-1, W, H, C])    

    # First convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv1',
                           filters=64, kernel_size=(11,5),
                           padding='same', activation=tf.nn.relu)
    net = tf.layers.max_pooling2d(inputs=net, pool_size=(8,4), strides=1)
    #net = tf.nn.lrn(input=net, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    # Second convolutional layer.
    net = tf.layers.conv2d(inputs=net, name='layer_conv2',
                           filters=200, kernel_size=(5,3),
                           padding='same', activation=tf.nn.relu)
    #net = tf.nn.lrn(input=net, depth_radius=4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')
    net = tf.layers.max_pooling2d(inputs=net, pool_size=(4,3), strides=1)    

    # Flatten to a 2-rank tensor.
    #net = tf.contrib.layers.flatten(net)
    # Eventually this should be replaced with:
    net = tf.layers.flatten(net)

    # First fully-connected / dense layer.
    # This uses the ReLU activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc1',
                          units=1024, activation=tf.nn.relu)    
    
    # Second fully-connected / dense layer
    net = tf.layers.dense(inputs=net, name='layer_fc2',
                          units=2048, activation=tf.nn.relu)  
    
   
    # This is the last layer so it does not use an activation function.
    net = tf.layers.dense(inputs=net, name='layer_fc3',
                          units=47)

    # Logits output of the neural network.
    logits = net

    # Softmax output of the neural network.
    y_pred = tf.nn.softmax(logits=logits)
    
    # Classification output of the neural network.
    y_pred_cls = tf.argmax(y_pred, axis=1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        # If the estimator is supposed to be in prediction-mode
        # then use the predicted class-number that is output by
        # the neural network. Optimization etc. is not needed.
        spec = tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=y_pred_cls)
    else:
        # Otherwise the estimator is supposed to be in either
        # training or evaluation-mode. Note that the loss-function
        # is also required in Evaluation mode.
        
        # Define the loss-function to be optimized, by first
        # calculating the cross-entropy between the output of
        # the neural network and the true labels for the input data.
        # This gives the cross-entropy for each image in the batch.
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                                       logits=logits)

        # Reduce the cross-entropy batch-tensor to a single number
        # which can be used in optimization of the neural network.
        loss = tf.reduce_mean(cross_entropy)

        # Define the optimizer for improving the neural network.
        optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])

        # Get the TensorFlow op for doing a single optimization step.
        train_op = optimizer.minimize(
            loss=loss, global_step=tf.train.get_global_step())

        # Define the evaluation metrics,
        # in this case the classification accuracy.
        metrics = \
        {
            "accuracy": tf.metrics.accuracy(labels, y_pred_cls)
        }

        # Wrap all of this in an EstimatorSpec.
        spec = tf.estimator.EstimatorSpec(
            mode=mode,
            loss=loss,
            train_op=train_op,
            eval_metric_ops=metrics)
        
    return spec

In [17]:
params = {"learning_rate": 1e-4}

In [18]:
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,
                               model_dir="./checkpoints_tutorial17-2/")

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
model.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


In [ ]:
result = model.evaluate(input_fn=test_input_fn)

In [ ]:
result

In [ ]:
print("Classification accuracy: {0:.2%}".format(result["accuracy"]))

In [ ]:
predictions = model.predict(input_fn=predict_input_fn)

In [ ]:
cls_pred = np.array(list(predictions))
cls_pred

In [ ]:
y[:20]

In [ ]:
************************************************************************************************

In [ ]:
iter = ds.make_one_shot_iterator()
e = iter.get_next()
#print(features.shape)
with tf.Session() as sess:
    print(sess.run(e)) 
    


In [ ]:
iter = ds.make_one_shot_iterator()
features, labels = iter.get_next()


In [ ]:
feature_columns = tf.feature_column.numeric_column(features, shape=[370,24,3])
type(feature_columns)

In [ ]:
def generate_input_fn(filenames, mode=tf.estimator.ModeKeys.EVAL, batch_size=1):
  """Input function for Estimator API."""
  def _input_fn():
    dataset = ds #tf.data.TFRecordDataset(filenames=filenames)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    #dataset=dataset.zip(dataset)
    dataset = dataset.map(parse_record)

    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(2 * batch_size)
    #dataset = dataset.map(parse_record)

    images, labels = dataset.make_one_shot_iterator().get_next()
    

    
    features = {'images': images}
    return features, labels

  return _input_fn

In [ ]:
def get_feature_columns():
  """Define feature columns."""
  feature_columns = {
    'images': tf.feature_column.numeric_column(
        'images', (370, 24, 3)),
  }
  return feature_columns

In [ ]:
def serving_input_fn():
  """Define serving function."""
  receiver_tensor = {
      'images': tf.placeholder(shape=[None, 370, 24, 3], dtype=tf.float32)
  }
  features = {
      'images':  receiver_tensor['images']
  }
  return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)

In [ ]:
def inference(images):
  # 1st Convolutional Layer
  conv1 = tf.layers.conv2d(
      inputs=images, filters=64, kernel_size=[5, 5], padding='same',
      activation=tf.nn.relu, name='conv1')
  pool1 = tf.layers.max_pooling2d(
      inputs=conv1, pool_size=[3, 3], strides=2, name='pool1')
  norm1 = tf.nn.lrn(
      pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

  # 2nd Convolutional Layer
  conv2 = tf.layers.conv2d(
      inputs=norm1, filters=64, kernel_size=[5, 5], padding='same',
      activation=tf.nn.relu, name='conv2')
  norm2 = tf.nn.lrn(
      conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
  pool2 = tf.layers.max_pooling2d(
      inputs=norm2, pool_size=[3, 3], strides=2, name='pool2')

  # Flatten Layer
  shape = pool2.get_shape()
  pool2_ = tf.reshape(pool2, [-1, shape[1]*shape[2]*shape[3]])

  # 1st Fully Connected Layer
  dense1 = tf.layers.dense(
      inputs=pool2_, units=384, activation=tf.nn.relu, name='dense1')

  # 2nd Fully Connected Layer
  dense2 = tf.layers.dense(
      inputs=dense1, units=192, activation=tf.nn.relu, name='dense2')

  # 3rd Fully Connected Layer (Logits)
  logits = tf.layers.dense(
      inputs=dense2, units=47, activation=tf.nn.relu, name='logits')

  return logits

In [ ]:
def model_fn(features, labels, mode, params):
  # Create the input layers from the features
  feature_columns = list(get_feature_columns().values())

  images = tf.feature_column.input_layer(
    features=features, feature_columns=feature_columns)

  images = tf.reshape(
    images, shape=(-1, 370, 24, 3))

  # Calculate logits through CNN
  logits = inference(images)

  if mode in (tf.estimator.ModeKeys.PREDICT, tf.estimator.ModeKeys.EVAL):
    predicted_indices = tf.argmax(input=logits, axis=1)
    probabilities = tf.nn.softmax(logits, name='softmax_tensor')

  if mode in (tf.estimator.ModeKeys.TRAIN, tf.estimator.ModeKeys.EVAL):
    global_step = tf.train.get_or_create_global_step()
    label_indices = tf.argmax(input=labels, axis=1)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=labels, logits=logits)
    tf.summary.scalar('cross_entropy', loss)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {
        'classes': predicted_indices,
        'probabilities': probabilities
    }
    export_outputs = {
        'predictions': tf.estimator.export.PredictOutput(predictions)
    }
    return tf.estimator.EstimatorSpec(
        mode, predictions=predictions, export_outputs=export_outputs)

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss, global_step=global_step)
    return tf.estimator.EstimatorSpec(
        mode, loss=loss, train_op=train_op)

  if mode == tf.estimator.ModeKeys.EVAL:
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(label_indices, predicted_indices)
    }
    return tf.estimator.EstimatorSpec(
        mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
def train_and_evaluate():
  #model_dir = os.path.join(FLAGS.output_dir, FLAGS.model_name)

  # Create estimator from Keras model.
  estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      model_dir="./tmp/model",
      config=tf.estimator.RunConfig(
          # save_checkpoints_steps=FLAGS.save_checkpoints_steps,
          keep_checkpoint_max=5,
          tf_random_seed=13))

  # Profile Hook.
  profile_hook = tf.train.ProfilerHook(
      save_steps=True,
      output_dir="./tmp/model",
      show_dataflow=True,
      show_memory=True)

  # Specify training data paths, batch size and max steps.
  train_spec = tf.estimator.TrainSpec(
      input_fn=generate_input_fn(filenames="./tmp/model/train",
                                    mode=tf.estimator.ModeKeys.TRAIN,
                                    batch_size=28),
      max_steps=100,
      hooks=[profile_hook]
  )

  # Currently (2017.12.14) the latest tf only support exporter with keras models.
  exporter = tf.estimator.LatestExporter(
      name="./tmp/model/export", serving_input_receiver_fn=serving_input_fn,
      assets_extra=None, as_text=False, exports_to_keep=5)

  # Specify validation data paths, steps for evaluation and exporter specs
  eval_spec = tf.estimator.EvalSpec(
      input_fn=generate_input_fn(filenames="./tmp/model/eval",
                                    mode=tf.estimator.ModeKeys.EVAL,
                                    batch_size=28),
      steps=1,
      name=None,
      hooks=None,
      exporters=exporter # Iterable of Exporters, or single one or None.
  )

  tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)


In [ ]:
def parse_record(serialized_example):
  """Parsing CIFAR-10 dataset that is saved in TFRecord format."""
  features = tf.parse_single_example(
    serialized_example,
    features={
      'image': tf.FixedLenFeature([], tf.string),
      'label': tf.FixedLenFeature([], tf.int64),
    })

  image = tf.decode_raw(features['image'], tf.uint8)
  image.set_shape([3 * 370 * 24])
  image = tf.reshape(image, [3, 370, 24])
  image = tf.cast(tf.transpose(image, [1, 2, 0]), tf.float32)

  label = tf.cast(features['label'], tf.int32)
  label = tf.one_hot(label, 47)

  return image, label

In [ ]:
train_and_evaluate()

In [ ]:


estimator = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=47,
    dropout=0.1,
    model_dir="./tmp/model"
)



In [ ]:
ds.data

In [ ]:
ds.output_shapes

In [ ]:
ds.output_classes

In [ ]:
###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



def main():


  feature_columns = [tf.feature_column.numeric_column("x", shape=[370,24,3])]
  print(feature_columns)

  # Build 3 layer DNN with 10, 20, 10 units respectively.
  classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=47,
                                          model_dir="/tmp/model2")
  print(classifier)

  train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X[20:]},
    y=y[20:],
    num_epochs=None,
    shuffle=True)

  
  
  # Train model.
  classifier.train(input_fn=train_input_fn, steps=100)
  print('trained')
  return classifier

def evalf(classifier):
  
  print('start eval')
  
  test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X[:2]},
    y=y[:2],
    num_epochs=None,
    shuffle=False)

  print('start eval2')
  # Evaluate accuracy.
  accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
  print(accuracy_score)
  print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

 
cl=main()
print('trained2')
#evalf(cl)

In [ ]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X[:2]},
    y=y[:2],
    num_epochs=None,
    shuffle=False)


cl.evaluate(input_fn=test_input_fn)

In [ ]:
c=cl.predict(X[3])
print(c)

In [ ]:
cl

In [ ]:
pred_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": X[3]}

cl.predict((input_fn=test_input_fn))


In [ ]:
# Wrapping all together -> Switch between train and test set
EPOCHS = 10
BATCH_SIZE = 16
# # create a placeholder to dynamically switch between batch sizes
# # batch_size = tf.placeholder(tf.int64)
# # x, y = tf.placeholder(tf.float32, shape=[None,2]), tf.placeholder(tf.float32, shape=[None,1])
# # dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).repeat()
# # # using two numpy arrays
# # train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
# # test_data = (np.random.sample((20,2)), np.random.sample((20,1)))

# # n_batches = len(train_data[0]) // BATCH_SIZE
# # iter = dataset.make_initializable_iterator()
# # features, labels = iter.get_next()
# # make a simple model
net = tf.layers.dense(features, 8, activation=tf.tanh) # pass the first value from iter.get_next() as input
net = tf.layers.dense(net, 8, activation=tf.tanh)
prediction = tf.layers.dense(net, 1, activation=tf.tanh)
loss = tf.losses.mean_squared_error(prediction, labels) # pass the second value from iter.get_net() as label
train_op = tf.train.AdamOptimizer().minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # initialise iterator with train data
    sess.run(iter.initializer, feed_dict={ x: train_data[0], y: train_data[1], batch_size: BATCH_SIZE})
    print('Training...')
    for i in range(EPOCHS):
        tot_loss = 0
        for _ in range(n_batches):
            _, loss_value = sess.run([train_op, loss])
            tot_loss += loss_value
        print("Iter: {}, Loss: {:.4f}".format(i, tot_loss / n_batches))
    # initialise iterator with test data
    sess.run(iter.initializer, feed_dict={ x: test_data[0], y: test_data[1], batch_size: test_data[0].shape[0]})
    print('Test Loss: {:4f}'.format(sess.run(loss)))

In [ ]:
iter = ds.make_one_shot_iterator()
features, labels = iter.get_next()

In [ ]:
date='2011_09_26'
preprocess_filtering_data(date=date, serieses = [1, 2, 9], dir_path='/home/shahar_zuler/ProjectNexar')

# TO DO:
- split to train and validation in a smarter way

In [ ]:
def get_data(num_training=2000, num_validation=500, num_test=10000, date='2011_09_26'):
    # Load the raw KITTI data
    
    #X_train, y_train =preprocess_filtering_data(date=date, serieses = [1,2], dir_path='/home/shahar_zuler/ProjectNexar')
    #X_test, y_test = preprocess_data(date=date, state = 'Test', dir_path='/home/shahar_zuler/ProjectNexar')
    X_train = np.load('X_train.npy')
    y_train = np.load('y_train.npy')
    print (X_train.shape)
    X_test = X_train
    y_test = y_train
    
    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
#     mask = range(num_test)
#     X_test = X_test[mask]
#     y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
plt.hist(y_train, bins = 47)

In [ ]:
# clear old variables
tf.reset_default_graph()

# define our input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
X = tf.placeholder(tf.float32, [None, 370, 24, 3])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

# define model
def stixel_model(X,y,is_training):
    conv1 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=[11, 5], 
                             padding="same", use_bias=True, activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[8, 4], strides=(8, 4))
    conv2 = tf.layers.conv2d(inputs=pool1, filters=200, kernel_size=[5, 3], 
                             padding="same", use_bias=True, activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4, 3], strides=(4,3))
    pool2_flat = tf.layers.flatten(inputs=pool2)
    dense3 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout3 = tf.layers.dropout(inputs=dense3, rate=0.5, training=is_training)
    dense4 = tf.layers.dense(inputs=dropout3, units=2048, activation=tf.nn.relu)
    dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=is_training)
    y_out = tf.layers.dense(inputs=dropout4, units=47)
    return y_out

y_out = stixel_model(X,y,is_training)


#predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
 #     "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
  #    "probabilities": tf.nn.softmax(logits, name="softmax_tensor")

In [ ]:
# total_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(indices=y, depth=50, axis=-1), logits=y_out)
total_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.one_hot(indices=y, depth=47, axis=-1), logits=y_out)
#tf.summary.histogram('total_loss',total_loss) ####################
mean_loss = tf.reduce_mean(total_loss)




In [ ]:
# optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-2, momentum=0.9) 
# tf.train.MomentumOptimizer(learning_rate=1e-2, momentum=0.9)

'''# decay every 10000 steps with a base of 0.5:
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 1e-2
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_steps=10000, decay_rate=0.5, staircase=True)

# Passing global_step to minimize() will increment it at each step.
optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-2, momentum=0.9, decay=0)
train_step = optimizer.minimize(mean_loss, global_step=global_step)'''
# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate #### 5e-4
train_step = optimizer.minimize(mean_loss)

In [ ]:
"""
# batch normalization in tensorflow requires this extra dependency
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss)
"""

In [ ]:


def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=128, print_every=50,
              training=None, plot_losses=False):
    
    train_writer = tf.summary.FileWriter( './logs/2/train ', sess.graph) #TB
    # have tensorflow compute accuracy
    the_prediction = tf.argmax(predict,axis=1)
    correct_prediction = tf.equal(tf.argmax(predict,axis=1),y)
#     correct_prediction = correct = tf.equal(tf.argmax(tf.nn.sigmoid(predict),axis=1),y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.histogram('accuracy',accuracy) #TB
    tf.summary.scalar('accuracy',accuracy)
    tf.summary.scalar('mean_loss',mean_loss)

    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None

    variables = ['_', mean_loss,correct_prediction, the_prediction ,accuracy] #TB

    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute    
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses and accuracy
        correct = 0
        losses = []
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now}
            # get batch size
            actual_batch_size = yd[idx].shape[0]
            
            merge = tf.summary.merge_all() #TB
            variables[0] = merge #TB
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
  
            summary, loss, corr,pred, _ = session.run(variables,feed_dict=feed_dict) #TB
            print('pred: ', pred)
            train_writer.add_summary(summary, iter_cnt) #TB
    
            # aggregate performance stats
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Epoch {0}: Iteration {1}: with minibatch training loss = {2:.3g} and accuracy of {3:.2g}"\
                      .format(e, iter_cnt,loss,np.sum(corr)/actual_batch_size))
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2} Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    
    return total_loss,total_correct

In [ ]:
tensors_to_log = {"probabilities": "softmax_tensor"}

saver = tf.train.Saver()
sess = tf.Session()


sess.run(tf.global_variables_initializer())
saver.save(sess, './training_data/stixelnet_model.ckpt')
print('Training')
run_model(session=sess, predict=y_out, loss_val=mean_loss, Xd=features, yd=labels, 
          epochs=20, batch_size=128, print_every=5, training=train_step)




In [ ]:
print('Validation')
run_model(session=sess, predict=y_out, loss_val=mean_loss, Xd=X_val, yd=y_val, 
          epochs=1, batch_size=128)


In [ ]:
#from preprocessing.preprocess_func_v02_new import *
#preprocess_filtering_data(date='2011_09_26', serieses = [5], dir_path='/home/shahar_zuler/ProjectNexar')

In [ ]:
print('Validation2')
X5 = np.load('X_train_5.npy')
y5 = np.load('y_train_5.npy')
run_model(session=sess, predict=y_out, loss_val=mean_loss, Xd=X5, yd=y5, 
          epochs=1, batch_size=128)